In [1]:
import pysam

In [15]:
bampath = '/projects/ps-yeolab5/ekofman/Sammi/MouseBrainEF1A_SingleCell_EPR_batch2/filtered_possorted_ms_hippo_stamp_bam/filtered_keep_xf25_possorted_genome_with_header.bam_MD.bam'


In [16]:
samfile = pysam.AlignmentFile(bampath, "rb")

In [55]:
complements = {
    'A': 'T',
    'C': 'G',
    'G': 'C',
    'T': 'A'
}

def reverse_complement(input_seq):
    rev_input_seq = reversed(input_seq)
    new_seq = []
    for r in rev_input_seq:
        new_seq.append(complements.get(r))
        
    return ''.join(new_seq)

reverse_complement("AATTGC")

'GCAATT'

In [97]:
def get_positions_from_md_tag(md_tag):
    md_tag_parsed = []
    for c in md_tag:
        try:
            md_tag_parsed.append(str(int(c)))
        except Exception as e:
            md_tag_parsed.append('-')

    positions = []

    position_splitters = [int(i) for i in ''.join(md_tag_parsed).split('-')]
    for s in position_splitters:
        if len(positions) == 0:
            positions.append(s)
        else:
            positions.append(positions[-1] + s + 1)
    return positions


  

for read in samfile.fetch('16', 3495912, 3505912):
    md_tag = read.get_tag('MD')
    positions_replaced = get_positions_from_md_tag(md_tag)
    
    print('MD tag', md_tag)
    print(read.cigarstring)
    print(read.cigartuples)

    print('Reverse?', read.is_reverse)
    
    aligned_seq = reverse_complement(read.get_forward_sequence())
    
    
    
    indicated_aligned_seq = []
    for i, a in enumerate(aligned_seq):
        if i in positions_replaced:
            indicated_aligned_seq.append(a.upper())
        else:
            indicated_aligned_seq.append(a.lower())
    
            
    print('Forward aligned sequence\n', ''.join(indicated_aligned_seq))
    print('Reference sequence\n', read.get_reference_sequence().lower())
    print('barcode', read.get_tag('CR'))
    print()
    break

MD tag 14G3G25G2G26C24
18M2I81M
[(0, 18), (1, 2), (0, 81)]
Reverse? True
Forward aligned sequence
 ttattttagttcctTaaaTttttttttctggactacataagtgaTgtGaaaaagaattaagtggttagaacatgTaatttccacccatcaactatgtgtAt
Reference sequence
 ttattttagttcctgaaagttttttctggactacataagtgatgggagaaagaattaagtggttagaacatgtactttccacccatcaactatgtgtat
barcode CAGCACGGTTCCTAAG

